In [1]:
%pip install -qU langchain langchain-core langchain-community langchain-ollama langchain-groq python-dotenv pypdf langchain-pinecone pinecone-notebooks redis

Note: you may need to restart the kernel to use updated packages.


# Libraries

In [1]:
import os, getpass, textwrap, time, uuid
from dotenv import load_dotenv # type: ignore

from langchain.storage import InMemoryStore # type: ignore
# from langchain.vectorstores import Chroma # type: ignore
from langchain_core.prompts import ChatPromptTemplate # type: ignore
from langchain_community.callbacks import get_openai_callback # type: ignore
from langchain_core.runnables import RunnablePassthrough # type: ignore
from langchain_core.output_parsers import StrOutputParser # type: ignore

from langchain_groq import ChatGroq # type: ignore

In [2]:
# Load environment variables from .env file
load_dotenv()

True

# Helper Function

In [3]:
def pprint_docs(docs):
    print(f"\n{'-' * 70}\n".join([f"Document {i+1}:\n\n" + "\n".join(textwrap.wrap(d.page_content)) for i, d in enumerate(docs)]))

def pprint_result(result):
    print("Answer: " + "\n".join(textwrap.wrap(result)))

# Load PDF File

In [4]:
from langchain_community.document_loaders import PyPDFLoader # type: ignore

# load the PDF  
file_path = "data/5. Pedoman Menteri PANRB NO 3 Tahun 2024 Pedoman Tata Cara Pemantauan dan Evaluasi SPBE.pdf"
loader = PyPDFLoader(file_path)

documents = loader.load()

In [5]:
documents[1]

Document(metadata={'producer': 'Microsoft® Word 2019', 'creator': 'Microsoft® Word 2019', 'creationdate': '2024-07-15T14:04:57+07:00', 'author': 'Muthia N R', 'moddate': '2024-07-15T14:04:57+07:00', 'source': 'data/5. Pedoman Menteri PANRB NO 3 Tahun 2024 Pedoman Tata Cara Pemantauan dan Evaluasi SPBE.pdf', 'total_pages': 236, 'page': 1, 'page_label': '2'}, page_content='-2- \njdih.menpan.go.id \nUpaya untuk mendorong perkembangan keterpaduan penerapan SPBE \npada Instansi Pusat dan Pemerintah Daerah dilakukan melalui penerapan \nArsitektur SPBE. Arsitektur SPBE Nasional telah ditetapkan melalui Peraturan \nPresiden Nomor 132 Tahun 2022 tentang Arsitektu r SPBE Nasional, sehingga \npembangunan layanan digital pada Instansi Pusat dan Pemerintah Daerah \nharus selaras dengan Arsitektur SPBE Nasional. Selain itu, pada tahun 2023 \ntelah ditetapkan Peraturan Presiden Nomor 82 Tahun 2023 tentang Percepatan \nTransformasi Digital dan Keterpaduan Layanan Digital Nasional, yang menjadi \nsalah

# 2. Load Redis as Persistent store

In [ ]:
# run command: docker run -d -p 6379:6379 redis

In [6]:
from langchain.storage.redis import RedisStore

# Persistent store: Redis
redis_url = "redis://localhost:6379"
redis_store = RedisStore(redis_url=redis_url, namespace="spbebot_docs")

# Chunking the SPBE with Parent Document Retriever

In [7]:
from langchain_text_splitters import RecursiveCharacterTextSplitter # type: ignore

# Create a splitter for the parent documents
text_parent = RecursiveCharacterTextSplitter(chunk_size = 1500)

# Create a splitter for the child documents
# Note: child documents should be smaller than parent documents
text_child = RecursiveCharacterTextSplitter(chunk_size = 400)

# 3 Connect to Pinecone Vector Database

In [8]:
from pinecone import Pinecone, ServerlessSpec # type: ignore

if not os.getenv("PINECONE_API_KEY"):
    os.environ["PINECONE_API_KEY"] = getpass.getpass("Enter your Pinecone API key: ")

pinecone_api_key = os.environ.get("PINECONE_API_KEY")

pc = Pinecone(api_key=pinecone_api_key)

In [9]:
import time

index_name = "spbedoc1"

existing_indexes = [index_info["name"] for index_info in pc.list_indexes()]

if index_name not in existing_indexes:
    pc.create_index(
        name=index_name,
        dimension=768,
        metric="cosine",
        spec=ServerlessSpec(cloud="aws", region="us-east-1"),
    )
    while not pc.describe_index(index_name).status["ready"]:
        time.sleep(1)

index = pc.Index(index_name)

In [10]:
index

# 2. Load Embedding Model

In [11]:
from langchain_ollama import OllamaEmbeddings # type: ignore

# load embedding model
embeddings = OllamaEmbeddings(
    model="nomic-embed-text:latest",
)

In [12]:
from langchain_pinecone import PineconeVectorStore # type: ignore

# initialize the vectorstore
pinecone_store = PineconeVectorStore(index=index, embedding=embeddings)

In [13]:
from langchain.retrievers import ParentDocumentRetriever # type: ignore

# Create a parent document retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=pinecone_store,
    docstore=redis_store,
    child_splitter=text_child,
    parent_splitter=text_parent,
    search_type="mmr",
    search_kwargs={"k": 2, "score_threshold": 0.8}
)

In [17]:
for i, doc in enumerate(documents):
    try:
        parent_retriever.add_documents([doc])
        print(f"Dokumen {i+1}/{len(documents)} berhasil ditambahkan")
    except Exception as e:
        print(f"Gagal menambahkan dokumen {i+1}: {e}")

Gagal menambahkan dokumen 1: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 2: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 3: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 4: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 5: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 6: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 7: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 8: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 9: Invalid input of type: 'Document'. Convert to a bytes, string, int or float

KeyboardInterrupt: 

In [16]:
# Use internal split method to prepare
docs, full_docs = parent_retriever._split_docs_for_adding(documents)

# 8. Filter safe size chunks (<3500 chars for Pinecone)
MAX_CHUNK_SIZE = 3500
safe_docs = [doc for doc in docs if len(doc.page_content) < MAX_CHUNK_SIZE]
safe_full_docs = {}
for doc_id, doc in full_docs:
    if len(doc.page_content) < MAX_CHUNK_SIZE:
        doc.metadata["doc_id"] = doc_id  # 👈 tambahkan ini
        safe_full_docs[doc_id] = doc


# 9. Upload by batch
batch_size = 10
for i in range(0, len(safe_docs), batch_size):
    batch = safe_docs[i:i + batch_size]
    try:
        parent_retriever.vectorstore.add_documents(batch)
        parent_retriever.docstore.mset({
            doc.metadata["doc_id"]: doc for doc in safe_full_docs.values()
            if doc.metadata["doc_id"] in [d.metadata["doc_id"] for d in batch]
        })
        print(f"✅ Batch {i // batch_size + 1} uploaded successfully")
    except Exception as e:
        print(f"❌ Error on batch {i // batch_size + 1}: {e}")

❌ Error on batch 1: too many values to unpack (expected 2)
❌ Error on batch 2: too many values to unpack (expected 2)
❌ Error on batch 3: too many values to unpack (expected 2)
❌ Error on batch 4: too many values to unpack (expected 2)
❌ Error on batch 5: too many values to unpack (expected 2)
❌ Error on batch 6: too many values to unpack (expected 2)
❌ Error on batch 7: too many values to unpack (expected 2)
❌ Error on batch 8: too many values to unpack (expected 2)


KeyboardInterrupt: 

In [27]:
for i, doc in enumerate(documents):
    try:
        parent_retriever.add_documents([doc])
        print(f"Dokumen {i+1}/{len(documents)} berhasil ditambahkan")
    except Exception as e:
        print(f"Gagal menambahkan dokumen {i+1}: {e}")

Gagal menambahkan dokumen 1: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.
Gagal menambahkan dokumen 2: Invalid input of type: 'Document'. Convert to a bytes, string, int or float first.


KeyboardInterrupt: 

In [ ]:
# Assign unique IDs for each parent doc
for i, doc in enumerate(parent_docs):
    doc.metadata["parent_doc_id"] = f"parent-{uuid.uuid4()}"


# === 3. Split to child chunks and preserve mapping ===
child_docs = []
for parent_doc in parent_docs:
    children = text_child.split_documents([parent_doc])
    for child in children:
        child.metadata["parent_doc_id"] = parent_doc.metadata["parent_doc_id"]
    child_docs.extend(children)

In [15]:
from langchain.retrievers import ParentDocumentRetriever # type: ignore

# Create a parent document retriever
parent_retriever = ParentDocumentRetriever(
    vectorstore=pinecone_store,
    docstore=store_parent, 
    child_splitter=text_child,
    parent_splitter=text_parent,
    search_type="mmr",
    search_kwargs={"k": 2, "score_threshold": 0.85}
)

# add documents to vectorstore
parent_retriever.add_documents(documents, batchsize=5)

PineconeApiException: (400)
Reason: Bad Request
HTTP response headers: HTTPHeaderDict({'Date': 'Tue, 22 Apr 2025 14:35:18 GMT', 'Content-Type': 'application/json', 'Content-Length': '118', 'Connection': 'keep-alive', 'x-pinecone-request-latency-ms': '67768', 'x-pinecone-request-id': '3436596206937991162', 'x-envoy-upstream-service-time': '2', 'server': 'envoy'})
HTTP response body: {"code":11,"message":"Error, message length too large: found 4662596 bytes, the limit is: 4194304 bytes","details":[]}


In [ ]:
# Cek jumlah dokumen yang tersimpan
print(f"Jumlah dokumen dalam ChromaDB: {pinecone_store._collection.count()}")

In [ ]:

vectorstore = Chroma.from_documents(
    collection_name="spbe_chunk",
    documents=chunk_byMarkdown,
    embedding=embeddings,
    persist_directory ="./pinecone_spbe_db"
)   

In [8]:
# Initialize a vector store to storing the chunks
parent_vstore = Chroma(
    collection_name="rag_parent_kuhp",
    embedding_function=embeddings,
    persist_directory ="./chroma_kuhp_parent_db"
)

# Initialize in-memory storage for the parent chunks
store_parent = InMemoryStore()

NameError: name 'embeddings' is not defined

# 3. Retriever

In [ ]:
# make retriever

naive_retriever = vectorstore.as_retriever(
    search_type="mmr", search_kwargs={"score_threshold": 0.8, "k": 2}
    ) 

In [ ]:
liat = naive_retriever.invoke("berapa lama masa percobaan yang dijatuhkan hakim pada pidana mati dan memperhatikan apa saja pada pasal 100?")
len(liat)

# 4. Load LLM

In [ ]:
# Load environment variables from .env file
load_dotenv()

# Verify API key is loaded
if not os.getenv("GROQ_API_KEY"):
    raise ValueError("GROQ_API_KEY not found in .env file")


model = ChatGroq(
    model_name="llama-3.3-70b-versatile",
    temperature=0,
    groq_api_key=os.getenv("GROQ_API_KEY"),
)

# 5. Prompt Template

In [1]:
# Log: Initializing prompt template
print("Initializing prompt template...")

prompt_template_naive = """
Anda adalah asisten hukum virtual yang andal dan sangat terlatih dalam hukum Indonesia, khususnya dalam memberikan informasi berdasarkan **Undang-Undang Nomor 1 Tahun 2023 tentang Kitab Undang-Undang Hukum Pidana (KUHP). 

---

## 🎯 Tugas Anda:
1️⃣ Menjawab pertanyaan umum tentang KUHP** secara akurat, tanpa menambah atau mengurangi makna pasal-pasal yang ada.  
2️⃣ Menunjukkan pasal, bab, dan ayat yang relevan** dengan pertanyaan pengguna.  
3️⃣ Menampilkan isi pasal dan ayat jika diminta** oleh pengguna.  
4️⃣ Memberikan informasi tentang pidana denda berdasarkan Pasal 78 dan Pasal 79.  
5️⃣ Mencari pasal berdasarkan kata kunci, jika pengguna tidak menyebutkan nomor pasal.  
6️⃣ Menyediakan mode "Ringkasan" dan "Detail", sesuai kebutuhan pengguna.  
7️⃣ Menyarankan langkah hukum dalam kasus tertentu jika pengguna membutuhkan.  
8️⃣ Menampilkan jawaban dalam format yang lebih mudah dibaca (tabel/poin-poin).

---

## 💰 Ketentuan tentang Denda dalam KUHP:
- Pasal 78: Pidana denda adalah sejumlah uang yang wajib dibayar oleh terpidana berdasarkan putusan pengadilan.
- Pasal 79: Denda dikategorikan sebagai berikut:

| Kategori | Nominal Denda |
|--------------|------------------|
| Kategori I | Rp1.000.000 |
| Kategori II | Rp10.000.000 |
| Kategori III | Rp50.000.000 |
| Kategori IV | Rp200.000.000 |
| Kategori V | Rp500.000.000 |
| Kategori VI | Rp2.000.000.000 |
| Kategori VII | Rp5.000.000.000 |
| Kategori VIII | Rp50.000.000.000 |

- Jika ada perubahan nilai uang, besaran denda akan ditentukan dalam Peraturan Pemerintah.

---

## 📢 Tanggapan Anda harus mengikuti aturan berikut:
- Jika pengguna bertanya tentang suatu pasal, tampilkan nomor pasal, bab, dan ayat yang relevan, lalu tampilkan isi pasal tersebut.  
- Jika pertanyaan berkaitan dengan pidana denda, informasikan kategori denda berdasarkan Pasal 78 dan 79.  
- Jika pengguna hanya ingin ringkasan, berikan jawaban singkat. Jika mereka ingin detail, berikan isi pasal lengkap.  
- Jika pertanyaan tidak mencantumkan nomor pasal, cari berdasarkan kata kunci dalam KUHP.  
- Jika pertanyaan berkaitan dengan langkah hukum dalam kasus tertentu, sarankan langkah-langkah yang dapat diambil berdasarkan KUHP.  
- Jika pertanyaan tidak tercakup dalam KUHP, berikan respons berikut:
  - *"Maaf, pertanyaan tersebut tidak tercakup dalam Undang-Undang Nomor 1 Tahun 2023 tentang KUHP. Saya hanya dapat memberikan jawaban berdasarkan isi dari undang-undang tersebut."

---

## 🔍 Konteks yang diberikan:
{context}

## ❓ Pertanyaan pengguna:
{question}

## ✅ Jawaban Anda:
"""

# Log: Creating ChatPromptTemplate from template
print("Creating ChatPromptTemplate from template...")

prompt = ChatPromptTemplate.from_template(prompt_template_naive)

# Log: ChatPromptTemplate initialized successfully
print("ChatPromptTemplate initialized successfully.")

# Function Invoke

In [4]:
def do_retrieval(chain):
    for i in range(len(questions)):
        print("-" * 40)
        print(f"Pertanyaan: {questions[i]}\n")
        with get_openai_callback() as cb:
            pprint_result(chain.invoke(questions[i]))
            print(f'\nTotal Tokens: {cb.total_tokens}\n')

In [ ]:
naive_chain = (
    {"context": naive_retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

In [ ]:
do_retrieval(naive_chain)